In [133]:
PORT = 10038
NUM_PAQUETES =  1
NUM_TIPOS = 1
CONT = 0
N=15


In [134]:
"""
Esta clase define a los usuarios que esatbleceran comunicacion con el servidor. 
Cada usuario contiene un ID que servira para darle seguimiento y saber en que 
se ejecutan sus paquetes

"""

class Usuario():
    
    tipo =  "A"
    ID  = "NA"
    numPauqtes = 1
    timeSatat = 0
    message = "SIN MENSAJE"
    
    def __init__( self, tipo, numPaquetes):
        global CONT
        self.tipo = tipo
        self.paquetes =  list()
        self.timeSatat = 0
        self.ID =  CONT
        self.message = "User ID: " + str(self.ID)
        for i in range(numPaquetes):
            self.paquetes.append( Paquete( self.tipo ) )
            
        CONT += 1 #Se modifica la variable global para llevar la cuenta de numero de usuarios
        
        self.imprimirUsuario()
        
    def imprimirUsuario(self):
        print( "\n Usuario ID: {} \n\t Tipo: {} \n\t Paquetes: ".format( self.ID, self.tipo))
        for i in self.paquetes:
            i.imprimirPaquete()
            
            
            
class Paquete():
    
    tiempoEjecucion = 0 #/tiempo que el servidor quedara suspendido en alguno de sus threads
    tipo = "alpha"
    ownerType =  "A"
    
    def __init__(self, owner):
        if owner  ==  "A":
            self.tipo = "alpha"
            self.tiempoEjecucion = 1
        elif owner == "B":
            self.tipo = "beta"
            self.tiempoEjecucion = 2
        elif owner == "C":
            self.tipo = "gamma"
            self.tiempoEjecucion = 3
    
    def imprimirPaquete(self):
        print( "\n\t\t **Paquete tipo {}  ||   Paquete Tiempo: {}".format( self.tipo, self.tiempoEjecucion))


    
    

In [135]:
listaUsuarios = list()

for i in range(N):
    newUser = Usuario( "A", 1)
    listaUsuarios.append(newUser)
    


 Usuario ID: 0 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 1 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 2 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 3 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 4 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 5 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 6 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 7 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 8 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 9 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario ID: 10 
	 Tipo: A 
	 Paquetes: 

		 **Paquete tipo alpha  ||   Paquete Tiempo: 1

 Usuario

In [136]:
import socket
import sys
from joblib import Parallel, delayed
from math import sqrt

from socket import error as SocketError
import errno
import time


def RunRequest(usuario, responsesList, dataList, messageList, sockList):
    # Create a TCP/IP socket
    # Connect the socket to the port where the server is listening
    server_address = ('localhost', PORT)
    try:
        sockList[usuario.ID].connect(server_address)
        # Send data
        messageList[usuario.ID] = usuario.message
        print('\n Sending "%s"' % usuario.message )
        sockList[usuario.ID].send( str.encode( usuario.message ) )
        
        """
        Aqui deberia ir algo asi como:
        
        procesos = ""
        for paq in usuario.pauquetes:
            procesos +=  str(paq.tiempoEjecucion)
            
        sockList[usuario.ID].send( str.encode( procesos ) )
        
        Lo que se inetenta es que el server pueda leer los paquetes que debe recibir del
        usuario y cuanto tiempo debe tardar en procesar cada uno.
        
        Pensandolo bien en cada iteracion (lectura del servidor) solo se deberia enviar uno
        de los paquetes del usuario para simular el pooling e los siguientes pasos
        
        ==>
        try:
            paq = usuario.paquetes.pop()
            sockList[usuario.ID].send( str.encode( paq.tiempoEjecucion ) )

                
        else: 
            print( "Todos los pquetes han sido leido ") 
        """

        # Look for the response
        amount_received = 0
        amount_expected = 10000000
        responsesList[usuario.ID] = ''
        start=time.time()
        now=time.time()
        print("\nTRANSMISION:")

        while (amount_received < amount_expected) and (start+10>now):

            dataList[usuario.ID] = sockList[usuario.ID].recv(2048).decode('utf-8')

            amount_received += len(dataList[usuario.ID])

            responsesList[usuario.ID] += dataList[usuario.ID]
            now=time.time()
            
        print('************ Received "%s" ************ \n' % responsesList[usuario.ID])
        sockList[usuario.ID].close()
        return

    except SocketError as e:
        if e.errno != errno.ECONNRESET:
            f=0
            pr0int("\nError raro")
        else:
            f=0
            print("\nError reset")
    finally:
        sockList[usuario.ID].close()
        return
        print("DONE")
    sockList[usuario.ID].close()
    return

responsesList=[]
dataList=[]
messageList=[]
sockList=[]
for k in range(N):
	responsesList.append("")
	dataList.append("")
	messageList.append("")
	sockList.append(socket.socket(socket.AF_INET, socket.SOCK_STREAM))

    
Parallel(n_jobs=n, backend="threading")(delayed(RunRequest)(listaUsuarios[k], responsesList, dataList, messageList, sockList) for k in range(n))




 Sending "User ID: 7"
 Sending "User ID: 5"
 Sending "User ID: 0"
 Sending "User ID: 6"
 Sending "User ID: 2"
 Sending "User ID: 3"
 Sending "User ID: 11"
 Sending "User ID: 9"
 Sending "User ID: 4"
 Sending "User ID: 1"
 Sending "User ID: 10"
 Sending "User ID: 13"
 Sending "User ID: 8"
 Sending "User ID: 14"
 Sending "User ID: 12"















TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:
TRANSMISION:














************ Received "
	SERVIDOR: INICIANDO (self.port  47234): User ID: 7
	------SERVIDOR: CORRIENDO (47234)
		SERVIDOR: TERMINANDO (47234): User ID: 7" ************ 

************ Received "
	SERVIDOR: INICIANDO (self.port  47224): User ID: 2
	------SERVIDOR: CORRIENDO (47224)
		SERVIDOR: TERMINANDO (47224): User ID: 2" ************ 
************ Received "
	SERVIDOR: INICIANDO (self.port  47240): User ID: 11
	------SERVIDOR: CORRIEN

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [137]:
k

14

In [140]:
for i in dataList:
    print(i)






	SERVIDOR: INICIANDO (self.port  47248): User ID: 4
	------SERVIDOR: CORRIENDO (47248)




	SERVIDOR: INICIANDO (self.port  47252): User ID: 8

		SERVIDOR: TERMINANDO (47242): User ID: 9

	SERVIDOR: INICIANDO (self.port  47250): User ID: 10
	------SERVIDOR: CORRIENDO (47250)



	SERVIDOR: INICIANDO (self.port  47246): User ID: 13
	------SERVIDOR: CORRIENDO (47246)

	SERVIDOR: INICIANDO (self.port  47254): User ID: 14


In [132]:
for i in responsesList:
    print("\n")
    print(i)




	SERVIDOR: INICIANDO (self.port  38438): User ID: 0
	------SERVIDOR: CORRIENDO (38438)
		SERVIDOR: TERMINANDO (38438): User ID: 0



	SERVIDOR: INICIANDO (self.port  38422): User ID: 1
	------SERVIDOR: CORRIENDO (38422)
		SERVIDOR: TERMINANDO (38422): User ID: 1



	SERVIDOR: INICIANDO (self.port  38436): User ID: 2
	------SERVIDOR: CORRIENDO (38436)
		SERVIDOR: TERMINANDO (38436): User ID: 2



	SERVIDOR: INICIANDO (self.port  38424): User ID: 3
	------SERVIDOR: CORRIENDO (38424)
		SERVIDOR: TERMINANDO (38424): User ID: 3



	SERVIDOR: INICIANDO (self.port  38426): User ID: 4
	------SERVIDOR: CORRIENDO (38426)
		SERVIDOR: TERMINANDO (38426): User ID: 4



	SERVIDOR: INICIANDO (self.port  38430): User ID: 5
	------SERVIDOR: CORRIENDO (38430)
		SERVIDOR: TERMINANDO (38430): User ID: 5



	SERVIDOR: INICIANDO (self.port  38442): User ID: 6



	SERVIDOR: INICIANDO (self.port  38428): User ID: 7
	------SERVIDOR: CORRIENDO (38428)
		SERVIDOR: TERMINANDO (38428): User ID: 7



	SERVIDOR: 